In [1]:
# Import required modules
import torch
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import torchvision.models as models

from google.colab import drive

import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_curve, auc

In [2]:
# Mount drive and retrieve path
drive.mount('/content/drive')

train_path = '/content/drive/MyDrive/Parkinsons-DaTscan-master/Implementation/TRAIN_CROP/'
test_path = '/content/drive/MyDrive/Parkinsons-DaTscan-master/Implementation/TEST_CROP/'
val_path = '/content/drive/MyDrive/Parkinsons-DaTscan-master/Implementation/VAL_CROP/'

Mounted at /content/drive


In [3]:
# Define preprocessing transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [4]:
# Create train and test data loaders
batch_size = 30
num_epochs = 30

train_loader = torch.utils.data.DataLoader(ImageFolder(train_path, transform=transform), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(ImageFolder(test_path, transform=transform), batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(ImageFolder(val_path, transform=transform), batch_size=batch_size, shuffle=True)

In [5]:
# Load pre-trained vgg16 model, Freeze pre-trained layers and Modify last layer
vgg16 = models.vgg16(pretrained=True)

for param in vgg16.parameters():
    param.requires_grad = False

num_classes = 2
vgg16.classifier[-1] = torch.nn.Linear(4096, num_classes)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:01<00:00, 278MB/s]


In [ ]:
# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.parameters(), lr=0.001)

accuracy_list = []
loss_list = []

# Train the model
for epoch in range(num_epochs):
    running_loss = 0.0
    total_correct = 0
    total_samples = 0
    for i, (images, labels) in enumerate(train_loader, 0):
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = vgg16(images)
        _, predicted = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Update metrics utilities
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

        # Print statistics
        running_loss += loss.item()
        if i % 10 == 9:
            print('[Epoch %d, Batch %d] Loss: %.3f' %(epoch + 1, batch_size, running_loss / 10))
            running_loss = 0.0

    accuracy = 100 * total_correct / total_samples
    accuracy_list.append(accuracy)

    epoch_loss = running_loss / len(train_loader)
    loss_list.append(epoch_loss)

[Epoch 1, Batch 30] Loss: 0.605
[Epoch 2, Batch 30] Loss: 0.434
[Epoch 3, Batch 30] Loss: 0.419
[Epoch 4, Batch 30] Loss: 0.392


In [ ]:
# Plot the accuracy versus epoch
plt.plot(range(1, num_epochs + 1), accuracy_list)
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy vs. Epoch')
plt.show()

In [ ]:
# Plot the loss versus epoch
plt.plot(range(1, num_epochs + 1), loss_list)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss vs. Epoch')
plt.show()

In [ ]:
# Evaluate on validation set
correct = 0
total = 0
with torch.no_grad():
    for data in val_loader:
        images, labels = data
        outputs = vgg16(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on validation set: %d %%' % (100 * correct / total))

In [ ]:
# Evaluate on test set
correct = 0
total = 0
actual = []
prediction = []

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = vgg16(images)
        _, predicted = torch.max(outputs.data, 1)
        actual.extend(labels.tolist())
        prediction.extend(predicted.tolist())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on test set: %d %%' % (100 * correct / total))

In [ ]:
# Create confusion matrix on test set
confusion_mtx = confusion_matrix(actual, prediction)
ax = plt.axes()
sn.heatmap(confusion_mtx, annot=True,annot_kws={"size": 25}, cmap="Blues", ax = ax)
ax.set_title('Test Accuracy', size=14)
plt.show()

In [ ]:
# Calculate scores
TN, FP, FN, TP = confusion_mtx.ravel()

precision = TP / (TP + FP)
recall = TP / (TP + FN)
specificity = TN / (TN + FP)
f1 = 2 * (precision * recall) / (precision + recall)

print("Precision:", precision)
print("Recall (Sensitivity):", recall)
print("Specificity:", specificity)
print("F1 Score:", f1)

In [ ]:
print("Classification Report:")
report = classification_report(actual, prediction, target_names=["Yes", "No"])
print(report)

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(num_epochs, loss, label='Training')
plt.plot(epochs_range, val_loss, label='Validation')
plt.legend(loc="best")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.grid(which='major', color='#666666', linestyle='-')
plt.tight_layout()
plt.show()